In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

HEALTH_SRC = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset"
SRC_BASE = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/Auto_labelling"

# Create train/val/test folders
for folder in ["train", "val", "test"]:
    for subfolder in ["images", "labels"]:
        os.makedirs(os.path.join(HEALTH_SRC, folder, subfolder), exist_ok=True)

# Define classes for Drone health monitoring
classes = ["healthy", "missing_part", "propeller_crack", "wing_damage"]


In [3]:
# Map folder names to health classes
folder_map = {
    "fixed_wing": "healthy",
    "hybrid": "healthy",
    "multi_rotor": "healthy",
    "single_rotor": "healthy",
    "missing_part": "missing_part",
    "propeller_crack": "propeller_crack",
    "wing_damage": "wing_damage"
}

# Copy images and labels
for cls_folder, health_cls in folder_map.items():
    
    src_img_path = os.path.join(SRC_BASE, "images", cls_folder)
    src_lbl_path = os.path.join(SRC_BASE, "labels", cls_folder)

    if not os.path.exists(src_img_path):
        continue

    imgs = [i for i in os.listdir(src_img_path) if i.lower().endswith(('.jpg', '.png', '.jpeg'))]

    # Split train/val/test
    train_imgs, temp_imgs = train_test_split(imgs, test_size=0.3, random_state=42)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.3333, random_state=42)

    splits = {"train": train_imgs, "val": val_imgs, "test": test_imgs}

    for split, split_imgs in splits.items():
        for img_file in split_imgs:
            # Copy image
            src_img = os.path.join(src_img_path, img_file)
            dst_img = os.path.join(HEALTH_SRC, split, "images", img_file)
            shutil.copy2(src_img, dst_img)

            # Copy label
            lbl_file = img_file.rsplit('.', 1)[0] + ".txt"
            src_lbl = os.path.join(src_lbl_path, lbl_file)
            dst_lbl = os.path.join(HEALTH_SRC, split, "labels", lbl_file)
            if os.path.exists(src_lbl):
                shutil.copy2(src_lbl, dst_lbl)


In [8]:
import os

HEALTH_LABELS = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset/train/labels"

class_map = {
    "healthy": 0,
    "missing_part": 1,
    "propeller_crack": 2,
    "wing_damage": 3
}

for split in ["train", "val", "test"]:
    split_lbl_path = os.path.join("D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset", split, "labels")
    
    for lbl_file in os.listdir(split_lbl_path):
        lbl_path = os.path.join(split_lbl_path, lbl_file)
        
        if "missing_part" in lbl_file:
            new_class = 1
        elif "propeller_crack" in lbl_file:
            new_class = 2
        elif "wing_damage" in lbl_file:
            new_class = 3
        else:
            new_class = 0  # healthy drones
        
        # Rewrite file with new class
        with open(lbl_path, "r") as f:
            lines = f.readlines()
        
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:
                parts[0] = str(new_class)
                new_lines.append(" ".join(parts))
        
        # Save back
        with open(lbl_path, "w") as f:
            f.write("\n".join(new_lines))


In [ ]:
import os

HEALTH_IMG = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset/train/images"
HEALTH_LBL = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset/train/labels"

img_files = os.listdir(HEALTH_IMG)
lbl_files = os.listdir(HEALTH_LBL)

print("Images:", len(img_files))
print("Labels:", len(lbl_files))


Images: 1033
Labels: 980


In [12]:
import os
import shutil

IMG_DIR = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset/train/images"
LBL_DIR = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset/train/labels"

img_files = os.listdir(IMG_DIR)
lbl_files = os.listdir(LBL_DIR)

# Extract base names without extension
img_basenames = {os.path.splitext(f)[0] for f in img_files}
lbl_basenames = {os.path.splitext(f)[0] for f in lbl_files}

# Find images without labels
missing_labels = img_basenames - lbl_basenames
print("Images without labels:", len(missing_labels))

# Optionally, create empty label files for them
for base in missing_labels:
    lbl_path = os.path.join(LBL_DIR, base + ".txt")
    open(lbl_path, "w").close()  # empty file

print("Label files synced! Total labels now:", len(os.listdir(LBL_DIR)))


Images without labels: 0
Label files synced! Total labels now: 980


In [13]:
import yaml
HEALTH_BASE = "D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset"

data_yaml = {
    'names': ['healthy', 'missing_part', 'propeller_crack', 'wing_damage'],
    'nc': 4,
    'train': os.path.join(HEALTH_BASE, 'train/images').replace("\\","/"),
    'val': os.path.join(HEALTH_BASE, 'val/images').replace("\\","/")
}

with open(os.path.join(HEALTH_BASE, 'health_data.yaml'), 'w') as f:
    yaml.dump(data_yaml, f)

print("health_data.yaml ready for training Drone health detection")


health_data.yaml ready for training Drone health detection


In [ ]:
from ultralytics import YOLO
import os

health_model = YOLO("yolov8n.pt")

health_model.train(
    data=os.path.join(HEALTH_SRC, "health_data.yaml"),
    epochs=25,
    imgsz=640,
    batch=16,
    name="drone_health_model",
    project="runs/train",
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.4.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.7  Python-3.12.6 torch-2.9.1+cpu CPU (11th Gen Intel Core i5-1135G7 @ 2.40GHz)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/Projects/Aero-AI_DroneDeliverySystem/Data/drone_health_dataset\health_data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002092DA663F0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480